In [ ]:


from fastai.text import *
import html
from sklearn.metrics import confusion_matrix

DATA_PATH=Path('/scratch/ppachigo/3192771/fastai/data/')    ## edit path
DATA_PATH.mkdir(exist_ok=True)
#! curl -O http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
#! tar -xzfv aclImdb_v1.tar.gz -C {DATA_PATH}

BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

PATH=Path('/scratch/ppachigo/3192771/fastai/data/toxic/')

CLAS_PATH=Path('/scratch/ppachigo/3192771/fastai/data/toxic_clas/')
CLAS_PATH.mkdir(exist_ok=True)

LM_PATH=Path('/scratch/ppachigo/3192771/fastai/data/toxic_lm/')
LM_PATH.mkdir(exist_ok=True)





In [ ]:

########### piecewise model fitting ############

trn_labels_tx = trn_labels[:,0]
trn_labels_stx = trn_labels[:,1]
trn_labels_obs = trn_labels[:,2]
trn_labels_thrt = trn_labels[:,3]
trn_labels_ins = trn_labels[:,4]
trn_labels_idthr = trn_labels[:,5]


val_labels_tx = val_labels[:,0]
val_labels_stx = val_labels[:,1]
val_labels_obs = val_labels[:,2]
val_labels_thrt = val_labels[:,3]
val_labels_ins = val_labels[:,4]
val_labels_idthr = val_labels[:,5]



In [ ]:
print('classifier tokens')
itos = pickle.load((LM_PATH/'tmp'/'itos.pkl').open('rb'))
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
print(len(itos))




In [ ]:
for i in range(6):

    print('classifier forward: ', i )
 
    trn_clas = np.load(CLAS_PATH/'tmp'/'trn_ids.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/'val_ids.npy')

    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'trn_labels.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'val_labels.npy'))

    trn_labels = trn_labels[:,i]
    val_labels = val_labels[:,i]

    bptt,em_sz,nh,nl = 70,400,1150,3
    vs = len(itos)
    opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
    bs = 24


    min_lbl = trn_labels.min()
    trn_labels -= min_lbl
    val_labels -= min_lbl
    c=int(trn_labels.max())+1


    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    md = ModelData(PATH, trn_dl, val_dl)


    print("# part 1")
    dps = np.array([0.4, 0.5, 0.05, 0.3, 0.1])

    dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.5

    m = get_rnn_classifier(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=.25
    learn.metrics = [accuracy]


    lr=3e-3
    lrm = 2.6
    lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])


    lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

    wd = 1e-7
    wd = 0
    learn.load_encoder('lm1_enc')

    learn.freeze_to(-1)

    learn.lr_find(lrs/1000)
    learn.sched.plot()

    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

    learn.save(f'clas_0_lbl{i}')

    learn.load(f'clas_0_lbl{i}')

    learn.freeze_to(-2)

    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

    learn.save(f'clas_1_lbl{i}')

    learn.load(f'clas_1_lbl{i}')

    learn.unfreeze()

    learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr=(32,10))

    learn.sched.plot_loss()

    learn.save(f'clas_2_lbl{i}')

    learn.load(f'clas_2_lbl{i}')

    print('fin')

    predictions = np.argmax(learn.predict(), axis =1 )

    print(predictions, len(predictions)) 
    acc = (val_labels == predictions).mean()

    print('Accuracy = ', acc)
    print('Confusion matrix = ', confusion_matrix(val_labels, predictions))

print('complete forward')







In [ ]:
for i in range(6):

    print('classifier backward: ', i )


    trn_clas = np.load(CLAS_PATH/'tmp'/'trn_ids_bwd.npy')
    val_clas = np.load(CLAS_PATH/'tmp'/'val_ids_bwd.npy')

    trn_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'trn_labels.npy'))
    val_labels = np.squeeze(np.load(CLAS_PATH/'tmp'/'val_labels.npy'))
     
    trn_labels = trn_labels[:,i]
    val_labels = val_labels[:,i]


    bptt,em_sz,nh,nl = 70,400,1150,3
    vs = len(itos)
    opt_fn = partial(optim.Adam, betas=(0.8, 0.99))
    bs = 24

    min_lbl = trn_labels.min()
    trn_labels -= min_lbl
    val_labels -= min_lbl
    c=int(trn_labels.max())+1


    trn_ds = TextDataset(trn_clas, trn_labels)
    val_ds = TextDataset(val_clas, val_labels)
    trn_samp = SortishSampler(trn_clas, key=lambda x: len(trn_clas[x]), bs=bs//2)
    val_samp = SortSampler(val_clas, key=lambda x: len(val_clas[x]))
    trn_dl = DataLoader(trn_ds, bs//2, transpose=True, num_workers=1, pad_idx=1, sampler=trn_samp)
    val_dl = DataLoader(val_ds, bs, transpose=True, num_workers=1, pad_idx=1, sampler=val_samp)
    md = ModelData(PATH, trn_dl, val_dl)



    # part 1
    dps = np.array([0.4, 0.5, 0.05, 0.3, 0.1])

    dps = np.array([0.4,0.5,0.05,0.3,0.4])*0.5

    m = get_rnn_classifier(bptt, 20*70, c, vs, emb_sz=em_sz, n_hid=nh, n_layers=nl, pad_token=1,
              layers=[em_sz*3, 50, c], drops=[dps[4], 0.1],
              dropouti=dps[0], wdrop=dps[1], dropoute=dps[2], dropouth=dps[3])

    opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

    learn = RNN_Learner(md, TextModel(to_gpu(m)), opt_fn=opt_fn)
    learn.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
    learn.clip=.25
    learn.metrics = [accuracy]


    lr=3e-3
    lrm = 2.6
    lrs = np.array([lr/(lrm**4), lr/(lrm**3), lr/(lrm**2), lr/lrm, lr])


    lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

    wd = 1e-7
    wd = 0
    learn.load_encoder('lm1_enc_bwd')

    learn.freeze_to(-1)

    learn.lr_find(lrs/1000)
    learn.sched.plot()

    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

    learn.save(f'clas_0_bwd_lbl{i}')

    learn.load(f'clas_0_bwd_lbl{i}')

    learn.freeze_to(-2)

    learn.fit(lrs, 1, wds=wd, cycle_len=1, use_clr=(8,3))

    learn.save(f'clas_1_bwd_lbl{i}')

    learn.load(f'clas_1_bwd_lbl{i}')

    learn.unfreeze()

    learn.fit(lrs, 1, wds=wd, cycle_len=10, use_clr=(32,10))

    learn.sched.plot_loss()

    learn.save(f'clas_2_bwd_lbl{i}')

    learn.load(f'clas_2_bwd_lbl{i}')

    print('fin')

    predictions = np.argmax(learn.predict(), axis =1 )

    print(predictions, len(predictions)) 
    acc = (val_labels == predictions).mean()

    print('Accuracy = ', acc)
    print('Confusion matrix = ', confusion_matrix(val_labels, predictions))

print('complete backward')



